# Проект по SQL

**Дано:** 

база данных крупного сервиса по чтению книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

**Цель**: оценит базу данных с точки зрения ее ценности.

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
for i in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = ''' SELECT * FROM {}; 
    '''.format(i) 

    request = pd.io.sql.read_sql(query, con = engine) 
    display(request.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица **books** - cодержит данные о книгах:

    * book_id — идентификатор книги;
    * author_id — идентификатор автора;
    * title — название книги;
    * num_pages — количество страниц;
    * publication_date — дата публикации книги;
    * publisher_id — идентификатор издателя.

Таблица **authors** - cодержит данные об авторах:

    * author_id — идентификатор автора;
    * author — имя автора.

Таблица **publishers** - cодержит данные об издательствах:

    * publisher_id — идентификатор издательства;
    * publisher — название издательства;

Таблица **ratings** - cодержит данные о пользовательских оценках книг:

    * rating_id — идентификатор оценки;
    * book_id — идентификатор книги;
    * username — имя пользователя, оставившего оценку;
    * rating — оценка книги.

Таблица **reviews** - cодержит данные о пользовательских обзорах:

    * review_id — идентификатор обзора;
    * book_id — идентификатор книги;
    * username — имя автора обзора;
    * text — текст обзора.

# 1я задача

In [5]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > DATE('2000-01-01');
''' 
pd.io.sql.read_sql(query, con = engine)

,count
0,819


819 книг вышло после 2020-01-01

# 2я задача

In [10]:
query = '''
SELECT b.title,
       b.book_id,
    t2.review_cnt AS review_cnt,
    t1.rating_avg AS rating_avg
FROM books AS b 
LEFT JOIN (SELECT book_id,
                  AVG(rating) AS rating_avg
           FROM ratings
           GROUP BY book_id
                ) AS t1 ON t1.book_id = b.book_id
LEFT JOIN (SELECT book_id,
                  COUNT(review_id) AS review_cnt
          FROM reviews
          GROUP BY book_id
                ) AS t2 ON t2.book_id = b.book_id
ORDER BY review_cnt DESC;
'''                
pd.io.sql.read_sql(query, con = engine)




,title,book_id,review_cnt,rating_avg
0,The Natural Way to Draw,808,NaN,3.000000
1,Disney's Beauty and the Beast (A Little Golden...,191,NaN,4.000000
2,Essential Tales and Poems,221,NaN,4.000000
3,Anne Rice's The Vampire Lestat: A Graphic Novel,83,NaN,3.666667
4,Leonardo's Notebooks,387,NaN,4.000000
...,...,...,...,...
995,Stone of Farewell (Memory Sorrow and Thorn #2),599,1.0,5.000000
996,Debt of Honor (Jack Ryan #7),187,1.0,3.000000
997,Color: A Natural History of the Palette,156,1.0,4.333333
998,Winter Prey (Lucas Davenport #5),984,1.0,4.500000


Для каждой книги посчитали количество обзоров и средний рейтинг.

# 3я задача

In [7]:
query = '''
SELECT publisher
FROM publishers
WHERE publisher_id = 
     (SELECT publisher_id
     FROM books 
     WHERE num_pages > 50
     GROUP BY publisher_id
     ORDER BY COUNT(book_id) DESC
     LIMIT 1);
''' 
pd.io.sql.read_sql(query, con = engine)

,publisher
0,Penguin Books


Определили издательство - Penguin Books, которое выпустило наибольшее число книг толще 50 страниц (исключили брошюры).

# 4я задача

In [8]:
query = '''
SELECT a.author,
       AVG(ra.rating) as rating_avg
FROM authors AS a
LEFT JOIN books AS b ON b.author_id=a.author_id
LEFT JOIN ratings AS ra ON ra.book_id=b.book_id
WHERE b.book_id IN (SELECT  book_id
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating) > 50)
GROUP BY a.author
ORDER BY AVG(ra.rating) DESC
LIMIT 1;
'''

pd.io.sql.read_sql(query, con = engine) 

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг — Дж.Роулинг, учитывались только книги с 50 и более оценками.

# 5я задача

In [9]:
query = '''
WITH t AS  (SELECT username,
           COUNT(review_id) AS review_cnt
           FROM reviews
           WHERE username IN (SELECT username
                              FROM ratings
                              GROUP BY username
                              HAVING COUNT(rating) > 50)
           GROUP BY username)
    
SELECT AVG(review_cnt)
FROM t
'''

pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.333333


Посчитали среднее количество обзоров  - 24.33 - от пользователей, которые поставили больше 50 оценок.

# Вывод

База данных довольно объемна: содержит 819 книг, выпущенных после 2020-01-01. Всего книг в базе -1000. Есть довольно активные пользователи приложения, которые оставляют отзывы и пишут обзоры. Ну и, конечно, есть книги самого популярного автора столетия -Джоан Роулинг.

База живая, объемная, но для успешных будущих продаж должна постоянно пополняться новыми книгами и стимулировать читателей оставлять отзывы и обзоры на прочитанные книги.